### 📊 Dashboard SQL Code forPower BI / Databricks SQL

### 🏥 1️⃣ Admissions Dashboard

🔹 KPI – Total Admissions

In [0]:
%sql
SELECT
    COUNT(DISTINCT encounter_id) AS total_admissions
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions;


### 🔹 KPI – Average Length of Stay

In [0]:
%sql
SELECT
    ROUND(AVG(length_of_stay_days), 2) AS avg_los_days
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
WHERE length_of_stay_days IS NOT NULL;


### 🔹 Admissions Trend (Monthly)

In [0]:
%sql
SELECT
    admit_year,
    admit_month,
    COUNT(encounter_id) AS admissions
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
GROUP BY admit_year, admit_month
ORDER BY admit_year, admit_month;


### 🔹 Admissions by Type

In [0]:
%sql
SELECT
    admission_type,
    COUNT(*) AS admissions
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
GROUP BY admission_type
ORDER BY admissions DESC;


### 🔹 Admissions by Gender

In [0]:
%sql
SELECT
    p.gender,
    COUNT(a.encounter_id) AS admissions
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions a
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_patient p
  ON a.patient_id = p.patient_id
GROUP BY p.gender;


## 2️⃣ Labs / Clinical Dashboard

###🔹 KPI – Total Observations

In [0]:
%sql
SELECT
    COUNT(*) AS total_observations
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_labs;


### 🔹 Average Lab Value by Observation

In [0]:
%sql
SELECT
    observation_name,
    ROUND(AVG(value), 2) AS avg_value
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_labs
WHERE value IS NOT NULL
GROUP BY observation_name
ORDER BY avg_value DESC;


### 🔹 Observation Trend Over Time

In [0]:
%sql
SELECT
    observation_name,
    observation_year,
    observation_month,
    AVG(value) AS avg_value
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_labs
GROUP BY observation_name, observation_year, observation_month
ORDER BY observation_name, observation_year, observation_month
LIMIT 100


🔹 Labs by Admission Type

In [0]:
%sql
SELECT
    admission_type,
    COUNT(*) AS observation_count
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_labs
GROUP BY admission_type;


## 💰 3️⃣ Financial / Cost Dashboard

###🔹 KPI – Total Healthcare Cost

In [0]:
%sql
SELECT
    ROUND(SUM(total_amount), 2) AS total_cost
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_costs;


### 🔹 Cost by Insurer

In [0]:
%sql
SELECT
    insurer,
    ROUND(SUM(total_amount), 2) AS total_cost
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_costs
GROUP BY insurer
ORDER BY total_cost DESC;


Databricks visualization. Run in Databricks to view.

### 🔹 Monthly Cost Trend

In [0]:
%sql
SELECT
    claim_year,
    claim_month,
    ROUND(SUM(total_amount), 2) AS monthly_cost
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_costs
GROUP BY claim_year, claim_month
ORDER BY claim_year, claim_month;


## 🩺 4️⃣ Diagnosis Dashboard

### 🔹 Most Common Diagnoses

In [0]:
%sql
SELECT
    diagnosis_name,
    COUNT(*) AS cases
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis
GROUP BY diagnosis_name
ORDER BY cases DESC;


### 🔹 Diagnosis Category Distribution

In [0]:
%sql
SELECT
    d.diagnosis_category,
    COUNT(*) AS cases
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis f
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis d
  ON f.diagnosis_key = d.diagnosis_key
GROUP BY d.diagnosis_category;


## ⚠️ 5️⃣ Risk & Utilization Dashboard

###🔹 KPI – High Risk Patients

In [0]:
%sql
SELECT
    COUNT(*) AS high_risk_patients
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_patient_risk
WHERE risk_score >= 50;


### 🔹 Risk Score Distribution

In [0]:
%sql
SELECT
    CASE
        WHEN risk_score >= 70 THEN 'High'
        WHEN risk_score >= 40 THEN 'Medium'
        ELSE 'Low'
    END AS risk_bucket,
    COUNT(*) AS patient_count
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_patient_risk
GROUP BY risk_bucket;


Databricks visualization. Run in Databricks to view.

### 🔹 Top 10 High-Risk Patients

In [0]:
%sql
SELECT
    patient_id,
    encounter_count,
    diagnosis_count,
    total_cost,
    risk_score
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_patient_risk
ORDER BY risk_score DESC
LIMIT 10;


Databricks visualization. Run in Databricks to view.


## 🧭 6️⃣ Executive Summary (Single View)



In [0]:
%sql
SELECT
    (SELECT COUNT(*) FROM angad_kumar91.fhir_healthcare_analytics_gold.dim_patient) AS total_patients,
    (SELECT COUNT(*) FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions) AS total_admissions,
    (SELECT ROUND(AVG(length_of_stay_days), 2) FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions) AS avg_los,
    (SELECT ROUND(SUM(total_amount), 2) FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_costs) AS total_cost;


In [0]:
%sql
SELECT 'Total Patients' AS metric, COUNT(*) AS value FROM angad_kumar91.fhir_healthcare_analytics_gold.dim_patient
UNION ALL
SELECT 'Total Admissions' AS metric, COUNT(*) AS value FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
UNION ALL
SELECT 'Avg LOS (Days)' AS metric, ROUND(AVG(length_of_stay_days), 2) AS value FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
UNION ALL
SELECT 'Total Cost' AS metric, ROUND(SUM(total_amount), 2) AS value FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_costs;
